In [408]:
import copy
import os

import pandas as pd
import polib
import requests

# Сгенерировать болванку для перевода

In [409]:
os.system('cd ../../../; xgettext -k_:1 -k__:1c,2 -kpgettext_noop:1c,2 -o ./lang/po/mon-data.po ./mon-data.h')

0

# Генерация файла с переводами всех существ в соответствующей контексту форме из Google таблиц

In [410]:
google_api_key = 

In [411]:
cells_range = 'A1:I1000'
creature_cells_range = f"Существа!{cells_range}"

url_template = f"https://sheets.googleapis.com/v4/spreadsheets/1uOzjsjzWLJRPYSPlL3v1urXRhC6mzg3L6DB0JjpicQo/values/{{}}?key={google_api_key}"
forms = 'Им', "Род", "Дат", "Вин", "Тв", "Пр"

In [412]:
creature_names_data = requests.get(url_template.format(creature_cells_range)).json()

In [413]:
creature_names_df = pd.DataFrame.from_records(
    creature_names_data['values'],
    columns=['#', 'msgid', *forms, "Пол"],
).set_index("msgid")[1:]
creature_names_df

,#,Им,Род,Дат,Вин,Тв,Пр,Пол
msgid,,,,,,,,
program bug,0,ошибка в программе,ошибки в программе,ошибке в программе,ошибку в программе,ошибкой в программе,ошибке в программе,f
bumblebee,1,шмель,шмеля,шмелю,шмеля,шмелём,шмеле,m
wood golem,2,деревянный голем,деревянного голема,деревянному голему,деревянного голема,деревянным големом,деревянном големе,m
ant larva,3,личинка муравья,личинки муравья,личинке муравья,личинку муравья,личинкой муравья,личинке муравья,f
laboratory rat,4,лабораторная крыса,лабораторной крысы,лабораторной крысе,лабораторную крысу,лабораторной крысой,лабораторной крысе,f
...,...,...,...,...,...,...,...,...
friendly sensed monster,650,дружелюбный учуянный монстр,дружелюбного учуянного монстра,дружелюбному учуянному монстру,дружелюбного учуянного монстра,дружелюбным учуянным монстром,дружелюбном учуянном монстре,m
trivial sensed monster,651,тривиальный учуянный монстр,тривиального учуянного монстра,тривиальному учуянному монстру,тривиального учуянного монстра,тривиальным учуянным монстром,тривиальном учуянном монстре,m
easy sensed monster,652,лёгкий учуянный монстр,лёгкого учуянного монстра,лёгкому учуянному монстру,лёгкого учуянного монстра,лёгким учуянным монстром,лёгком учуянном монстре,m


In [414]:
creature_contexts_cells_range = f"'Контексты существ'!{cells_range}"
creature_contexts_data = requests.get(url_template.format(creature_contexts_cells_range)).json()

In [415]:
creature_contexts_df = pd.DataFrame.from_records(
    creature_contexts_data['values'],
    columns=forms,
)[1:]
creature_contexts_df

,Им,Род,Дат,Вин,Тв,Пр
1,%s bites you for 10 damage with +3 dagger!!!,,The mighty blow completely misses %s!,The jackal bites %s for 10 damage with +3 dagg...,You collide with %s!,None
2,%s is blown up!,,Your bite misses %s.,You slice %s like an onion for 10 damage!!,Collided with %s caused by salamander mystic.,None
3,%s hits a closed door.,,A dart trap hits %s!,You kill %s!,Collided with the jackal caused by %s.,None
4,%s loses its grip on you.,,,Octopode loses its grip on %s.,None,None
5,"%s constrict the jackal for 15, but do no damage.",,,"The grasping roots constrict %s for 15, but do...",None,None
6,%s collides with you!,,,The captain's cutlass flashes! You lacerate %s!!,None,None
7,%s violently stops moving!,None,None,None,None,None
8,%s completely misses centaur captain skeleton!,None,None,None,None,None
9,"%s is constricted for 15, but do no damage.",None,None,None,None,None
10,%s slams into the wall!,None,None,None,None,None


In [416]:
def forms_gen():
    for form in forms:
        words_forms = creature_names_df[form]
        contexts = creature_contexts_df[form][lambda x: x != ''].dropna()
        yield form, contexts, words_forms
next(forms_gen())

('Им', 1          %s bites you for 10 damage with +3 dagger!!!
 2                                       %s is blown up!
 3                                %s hits a closed door.
 4                             %s loses its grip on you.
 5     %s constrict the jackal for 15, but do no damage.
 6                                 %s collides with you!
 7                            %s violently stops moving!
 8        %s completely misses centaur captain skeleton!
 9           %s is constricted for 15, but do no damage.
 10                              %s slams into the wall!
 Name: Им, dtype: object, msgid
 program bug                         ошибка в программе
 bumblebee                                        шмель
 wood golem                            деревянный голем
 ant larva                              личинка муравья
 laboratory rat                      лабораторная крыса
                                       ...             
 friendly sensed monster    дружелюбный учуянный монстр


In [417]:
import polib

In [418]:
translations_file = polib.POFile()

In [419]:
for form, contexts, words_forms in forms_gen():
    if form == forms[0]:
        for msgid in words_forms.index:
            translations_file.append(polib.POEntry(msgid=msgid, msgstr=words_forms[msgid]))
    for context in contexts:
        for msgid in words_forms.index:
            translations_file.append(polib.POEntry(msgid=msgid, msgstr=words_forms[msgid], msgctxt=context))
translations_file.save('./monsters_in_contexts.po')

# Сгенерировать правильные формы для слов, которые зависят от типа существа, к которому применяются

In [420]:
genus_cells_range = f"'Слова варьирующиеся от пола'!{cells_range}"
genus_data = requests.get(url_template.format(genus_cells_range)).json()

In [421]:
genus_df = pd.DataFrame.from_records(
    genus_data['values'],
    columns="msgid	m	f	n	Many".split('\t'),
)[1:].set_index('msgid')
genus_df

,m,f,n,Many
msgid,,,,
damaged,повреждён,повреждена,повреждено,повреждены
wounded,ранен,ранена,ранено,ранены
destroyed,уничтожен,уничтожена,уничтожено,уничтожены
dead,мёртв,метрва,мертво,мертвы


In [422]:
genus_translations_file = polib.POFile()

for creature_msgid in creature_names_df.index:
    creature_genus = creature_names_df.loc[creature_msgid]["Пол"]
    
    for genus_msgid in genus_df.index:
        genus_translations_file.append(polib.POEntry(msgid=genus_msgid, msgstr=genus_df.loc[genus_msgid][creature_genus], msgctxt=creature_msgid))

        
genus_translations_file.save('./monster_dependent_terms.po')